In [1]:
%%capture
!pip install datasets rouge-score nltk transformers sentencepiece

In [2]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers
import torch

In [3]:
torch_device ='cuda'
#torch_device = 'cpu' (run this line if no gpu attached)

In [4]:
#load original model
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(torch_device)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [5]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

In [8]:
prefix = "summarize: "

##XSUM

In [6]:
#load test dataset
test_dataset = datasets.load_dataset('xsum', split='test')

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [7]:
small_test = test_dataset.select(list(range(2000)))
small_test

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2000
})

In [9]:
# generate summary
batch_size = 64
max_input_length=512
max_target_length=64

def generate_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [10]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/32 [00:00<?, ?ba/s]

In [11]:
results

Dataset({
    features: ['summary', 'id', 'pred'],
    num_rows: 2000
})

In [12]:
metric = datasets.load_metric("rouge")
rouge_score = metric.compute(predictions=results["pred"], references=results["summary"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [13]:
rouge_mean

{'rouge1': 16.52647294070278,
 'rouge2': 2.1424770148206997,
 'rougeL': 13.026310453380137,
 'rougeLsum': 13.014565499615639}

In [14]:
#load fine-tuned model
!cp '/content/drive/My Drive/weights/t5-small-finetuned-xsum.zip' '/content'

In [15]:
!unzip t5-small-finetuned-xsum.zip

Archive:  t5-small-finetuned-xsum.zip
   creating: content/t5-small-finetuned-xsum/checkpoint-12500/
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/training_args.bin  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/scheduler.pt  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/scaler.pt  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/special_tokens_map.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/optimizer.pt  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/rng_state.pth  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/pytorch_model.bin  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/config.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/trainer_state.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/tokenizer.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-12500/tokenizer_config.json  


In [16]:
finetuned_model = T5ForConditionalGeneration.from_pretrained('content/t5-small-finetuned-xsum/checkpoint-12500').to(torch_device)

In [17]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [18]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/32 [00:00<?, ?ba/s]

In [19]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["summary"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [20]:
rouge_mean

{'rouge1': 16.52647294070278,
 'rouge2': 2.1424770148206997,
 'rougeL': 13.026310453380137,
 'rougeLsum': 13.014565499615639}

In [21]:
finetuned_rouge_mean

{'rouge1': 29.177938218660532,
 'rouge2': 8.239405261905869,
 'rougeL': 22.9168256855737,
 'rougeLsum': 22.915781545075912}

In [22]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['document'])
    print("\nActual Summary: %s" % results[i]['summary'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: Pakistan's telecoms regulator said the ban was no longer necessary because Google, which owns YouTube, had now launched a Pakistan-specific version.
YouTube has denied claims that the authorities can filter content.
Many young Pakistanis have welcomed the lifting of the ban but some activists want details of the deal with Google.
They say there should be greater transparency of the terms agreed between Google and the government.
A Pakistan Telecommunication Authority (PTA) official confirmed to the BBC that all internet service providers had been directed to open access to YouTube.
The Pakistan Telecommunication Company Ltd posted on its Facebook page on Monday: "Welcome Back YouTube".
Pakistan's ministry of information technology said: "Google has provided an online web process through which requests for blocking access of offending material can be made by the PTA to Google directly.
"Google/YouTube will accordingly restrict access to the said offending material for use

##CNN

In [23]:
#load test dataset
test_cnn = datasets.load_dataset('cnn_dailymail', '3.0.0', split='test')

Downloading:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


In [24]:
small_test = test_cnn.select(list(range(2000)))
small_test

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 2000
})

In [25]:
# generate summary
batch_size = 64
max_input_length=1024
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [26]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/32 [00:00<?, ?ba/s]

In [27]:
results

Dataset({
    features: ['highlights', 'id', 'pred'],
    num_rows: 2000
})

In [28]:
rouge_score = metric.compute(predictions=results["pred"], references=results["highlights"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [29]:
rouge_mean

{'rouge1': 22.98087498731323,
 'rouge2': 8.694945729673295,
 'rougeL': 18.365603689724217,
 'rougeLsum': 20.789266839492736}

In [30]:
#load fine-tuned model
!cp '/content/drive/My Drive/weights/t5-small-finetuned-cnn.zip' '/content'

In [31]:
!unzip t5-small-finetuned-cnn.zip

Archive:  t5-small-finetuned-cnn.zip
   creating: content/t5-small-finetuned-cnn/checkpoint-17500/
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/scheduler.pt  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/training_args.bin  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/scaler.pt  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/tokenizer_config.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/special_tokens_map.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/trainer_state.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/config.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/optimizer.pt  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/tokenizer.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/rng_state.pth  
  inflating: content/t5-small-finetuned-cnn/checkpoint-17500/pytorch_model.bin  


In [32]:
finetuned_model = T5ForConditionalGeneration.from_pretrained('content/t5-small-finetuned-cnn/checkpoint-17500').to(torch_device)

In [33]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [34]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/32 [00:00<?, ?ba/s]

In [35]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["highlights"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [36]:
rouge_mean

{'rouge1': 22.98087498731323,
 'rouge2': 8.694945729673295,
 'rougeL': 18.365603689724217,
 'rougeLsum': 20.789266839492736}

In [37]:
finetuned_rouge_mean

{'rouge1': 24.49523765780393,
 'rouge2': 10.382042005217992,
 'rougeL': 20.209801455007305,
 'rougeLsum': 22.417538707694106}

In [38]:
for i in range(0, 15):
    print("Original Text: %s" % small_test[i]["article"])
    print("\nActual Summary: %s" % results[i]["highlights"])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: (CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when "The Dukes of Hazzard's" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphras

##Wikihow

In [39]:
#load test dataset
test_wiki = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset", split='test')

Downloading:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Using custom data configuration all-6f5101161f12f62f


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikihow downloaded and prepared to /root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e. Subsequent calls will reuse this data.


In [40]:
small_test = test_wiki.select(list(range(2000)))
small_test

Dataset({
    features: ['text', 'headline', 'title'],
    num_rows: 2000
})

In [41]:
# generate summary
batch_size = 64
max_input_length=1024
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [42]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/32 [00:00<?, ?ba/s]

In [43]:
results

Dataset({
    features: ['headline', 'title', 'pred'],
    num_rows: 2000
})

In [44]:
rouge_score = metric.compute(predictions=results["pred"], references=results["headline"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [45]:
rouge_mean

{'rouge1': 16.250085847914136,
 'rouge2': 3.388503714433712,
 'rougeL': 12.771176162509363,
 'rougeLsum': 14.977192134767026}

In [46]:
#load fine-tuned model
!cp '/content/drive/My Drive/weights/t5-small-finetuned-wikihow.zip' '/content'

In [47]:
!unzip t5-small-finetuned-wikihow.zip

Archive:  t5-small-finetuned-wikihow.zip
   creating: content/t5-small-finetuned-wikihow/checkpoint-9000/
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/pytorch_model.bin  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/scaler.pt  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/tokenizer.json  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/training_args.bin  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/config.json  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/optimizer.pt  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/special_tokens_map.json  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/rng_state.pth  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/trainer_state.json  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/tokenizer_config.json  
  inflating: content/t5-small-finetuned-wikihow/checkpoint-9000/sched

In [48]:
finetuned_model = T5ForConditionalGeneration.from_pretrained('content/t5-small-finetuned-wikihow/checkpoint-9000').to(torch_device)

In [49]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [50]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/32 [00:00<?, ?ba/s]

In [51]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["headline"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [52]:
rouge_mean

{'rouge1': 16.250085847914136,
 'rouge2': 3.388503714433712,
 'rougeL': 12.771176162509363,
 'rougeLsum': 14.977192134767026}

In [53]:
finetuned_rouge_mean

{'rouge1': 25.010789542171274,
 'rouge2': 8.894838797771007,
 'rougeL': 21.15970417740787,
 'rougeLsum': 23.716052153743544}

In [54]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['text'])
    print("\nActual Summary: %s" % results[i]['headline'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: In general, dwarf hamsters have thickset bodies, large cheek pouches, and short tails. Make sure it does not have any nasal or eye discharge, nor any other signs of illness. Russian dwarf hamsters are particularly prone to diabetes. A hamster suffering from this condition will drink a lot of water and urinate more frequently than other dwarf hamsters.Be sure to consult your veterinarian if you suspect that your hamster has diabetes.
 A recently weaned or an extremely stressed out dwarf hamster may come down with a disease called “wet tail.” Your hamster experiences diarrhea — the excessive moisture from this causes its tail to become literally wet. Consult a veterinarian for a proper diagnosis and treatment.


Tyzzer’s disease causes diarrhea in young or stressed hamsters. This is a disease that needs veterinarian treatment. Certain antibiotics can cause and exacerbate this condition, so don’t treat your hamster on your own., Like dogs and cats, dwarf hamsters can suffer